In [1]:
import pandas as pd
df = pd.read_csv('/kaggle/input/fake-news/train.csv')

In [2]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [4]:
df = df.dropna()

In [5]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [6]:
X = df.drop('label', axis = 1)
y = df['label']

In [7]:
print(X.shape, y.shape)

(18285, 4) (18285,)


In [8]:
import tensorflow as tf
print(tf.__version__)

2024-06-12 10:04:00.512453: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-12 10:04:00.512559: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-12 10:04:00.659349: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2.15.0


In [9]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [10]:
voc_size = 5000

In [11]:
messages=X.copy()
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [16]:
messages.reset_index(inplace = True) # because we have removed the nan values

In [13]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [15]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]

In [17]:
onehot_repr[0]

[1597, 3509, 3857, 4004, 2644, 4103, 4731, 680, 1897, 4998]

In [18]:
sent_length = 20
padd_repr = pad_sequences(onehot_repr, padding = 'pre', maxlen = sent_length)

In [19]:
padd_repr[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1597,
       3509, 3857, 4004, 2644, 4103, 4731,  680, 1897, 4998], dtype=int32)

In [20]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
# Manually build the model with the input shape
model.build(input_shape=(None, sent_length))
print(model.summary())


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 20, 40)         │       200,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 200)            │       112,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           201 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 313,001 (1.19 MB)

 Trainable params: 313,001 (1.19 MB)

 Non-trainable params: 0 (0.00 B)

None


In [21]:
import numpy as np
X_final=np.array(padd_repr)
y_final=np.array(y)

In [22]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [33]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 1.0000 - loss: 4.0668e-06 - val_accuracy: 0.9102 - val_loss: 0.9576
Epoch 2/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 1.0000 - loss: 3.4005e-06 - val_accuracy: 0.9102 - val_loss: 0.9770
Epoch 3/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 1.0000 - loss: 2.5493e-06 - val_accuracy: 0.9095 - val_loss: 0.9940
Epoch 4/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 1.0000 - loss: 2.3411e-06 - val_accuracy: 0.9097 - val_loss: 1.0122
Epoch 5/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 1.0000 - loss: 1.7034e-06 - val_accuracy: 0.9102 - val_loss: 1.0300
Epoch 6/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 1.0000 - loss: 1.3667e-06 - val_accuracy: 0.9094 - val_loss: 1.0491
Epoch 7/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 1.0000 - loss: 1.2252e-06 - val_accuracy: 0.9094 - val_loss: 1.0661
Epoch 8/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 1.0000 - los

In [34]:
y_pred=model.predict(X_test)
y_pred=np.where(y_pred > 0.5, 1,0) ##AUC ROC Curve
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


array([[3103,  316],
       [ 228, 2388]])

In [35]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9098591549295775

In [36]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.88      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

